In [1]:
import pandas as pd

In [2]:
#obj = 'gs://input_data_amy_bkt1/input_data/Anonymized_Fermentation_Data_final.xlsx'
obj = open('/home/jupyter/aiplatform/data/Anonymized_Fermentation_Data_final.xlsx', 'rb')
data = pd.read_excel(obj,sheet_name='data') 
meta_data = pd.read_excel(obj,sheet_name='meta data')

In [3]:
data.head()

,Product,Purpose,experiment,run,Project_Name,run_label,Strain,strain_key,Feedstock_Parent1,Start_Time,...,Cap_Oil_Em_End__percent,Cap_PCV_End__percent,Cap_Dead_Cell_Layer_End__percent,Zeex9ieJAlt_end__g_L,Zeex9ieJ_mAU_sec_end__area,Zeex9ieJ_Screening_end__g_L,Zeex9ieJ_end__g_L,Zeex9ieJ_end__g_L.1,Zeex9ieJ_end__g_L.2,interval_type
0,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,14.53607,NaN,1.673583,0.197670,2.751213,0.000000,0.150839,NaN,Cumulative
1,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,18.90319,NaN,4.333049,0.285636,3.600460,0.000000,0.294163,NaN,Cumulative
2,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,19.13803,NaN,5.186557,0.289196,3.596406,0.000000,0.325491,NaN,Cumulative
3,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,26.28490,NaN,10.083622,0.394463,4.482558,0.054673,0.277906,0.0,Cumulative
4,Zeex9ieJ,MF,94CB02FEB7,9,Zeex9ieJ for All,12664-9,94CB02FEB7,9427397,m1098919,2021-04-09 15:52:34,...,0.0,18.40111,NaN,12.402302,0.500296,5.840546,0.122632,0.301150,0.0,Cumulative


In [4]:
meta_data.head()

,index,name,description,variable type,target
0,0,Product,name of product,metadata,0
1,1,Purpose,"Fermentation purpose MF, IF, SF etc",metadata,0
2,2,experiment,Experiment ID,independent,0
3,3,run,Run ID within experiment,metadata,0
4,4,Project_Name,Project name,metadata,0


In [5]:
print('The data contains {} samples and {} variables'.format(data.shape[0],data.shape[1]))
print('There are {} unique strains which are replicated or measured under different fermentation conditions.'.format(data['Strain'].nunique()))
print('The variables are comprised of a variety of fermentation process, physiological and biochemical parameters:')
display(meta_data['variable type'].value_counts())
print('Predictive modeling targets include: {}'.format(meta_data.query('target == 1').name.values))
display(data[['Run_Execution', 'Run_Performance']].apply(lambda x: x.value_counts()))

The data contains 1568 samples and 124 variables
There are 20 unique strains which are replicated or measured under different fermentation conditions.
The variables are comprised of a variety of fermentation process, physiological and biochemical parameters:


independent    63
metadata       53
dependent       5
category        3
Name: variable type, dtype: int64

Predictive modeling targets include: ['Run_Execution' 'Run_Performance' 'Product_Produced__g'
 'Titer_End__g_over_kg']


,Run_Execution,Run_Performance
alpha,1433.0,NaN
beta,135.0,NaN
delta,NaN,1058.0
gamma,NaN,510.0


In [6]:
#prepare data for analysis
#split out numeric from categorical varibles
var_type_filter = [x in ['independent'] for x in meta_data['variable type']]
var_dtype_filter = (data.dtypes == 'float64') | (data.dtypes == 'int64')

numeric_vars = (var_type_filter & var_dtype_filter).values
data[data.columns[numeric_vars]]

,Viable_Cell_Density__g_over_L,Product_Flux_Viable_Cells__mmol_over_gDCW_over_h,Growth_Rate_Viable_Cells__1_over_h,Death_Rate_Avg__1_over_h,Death_Rate_Avg_Viable_Cells__1_over_h,TRS_Flux_Viable_Cells__mmol_over_gDCW_over_h,TRS_Flux__mmol_over_gDCW_over_h,TRS_for_Other_Viable_Cells__mmol_over_gDCW_over_h,sOUR_Viable_Cells__mmol_over_gDCW_over_h,TRS_to_Maintenance_Flux__mmol_over_gDCW_over_h,...,Cap_Cane_PCV_End__percent,Cap_Feedstock_Solids_End__percent,Cap_Heavy_Em_End__percent,Cap_Light_Em_End__percent,Cap_Oil_Em_End__percent,Cap_PCV_End__percent,Cap_Dead_Cell_Layer_End__percent,Zeex9ieJAlt_end__g_L,Zeex9ieJ_mAU_sec_end__area,Zeex9ieJ_Screening_end__g_L
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.153653,...,0.57894,NaN,0.86059,0.00000,0.00000,14.53607,NaN,1.673583,0.197670,2.751213
1,NaN,NaN,NaN,NaN,NaN,NaN,0.451709,NaN,NaN,0.210791,...,0.59743,NaN,1.28676,0.00000,0.00000,18.90319,NaN,4.333049,0.285636,3.600460
2,NaN,NaN,NaN,NaN,NaN,NaN,0.307022,NaN,NaN,0.236642,...,0.34659,NaN,0.24111,0.96444,0.00000,19.13803,NaN,5.186557,0.289196,3.596406
3,NaN,NaN,NaN,NaN,NaN,NaN,0.420256,NaN,NaN,0.274503,...,0.88832,NaN,2.36358,0.00000,0.00000,26.28490,NaN,10.083622,0.394463,4.482558
4,NaN,NaN,NaN,NaN,NaN,NaN,0.405015,NaN,NaN,0.393885,...,2.50503,NaN,1.06696,1.76280,0.00000,18.40111,NaN,12.402302,0.500296,5.840546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,17.614806,0.049335,0.013222,0.003184,0.005342,1.403776,0.836621,1.119942,4.461783,0.667462,...,1.89140,NaN,0.48810,0.51861,36.15009,11.83649,NaN,10.791450,888.284510,14.138946
1564,14.819922,0.050276,0.009846,0.002457,0.004706,1.358599,0.709208,1.125262,4.787741,0.587403,...,2.37672,NaN,1.37429,0.00000,38.18917,10.31528,NaN,13.428900,953.769643,15.153186
1565,11.736481,0.059602,-0.005247,-0.001233,-0.003267,2.087119,0.787523,1.831997,6.264150,0.691259,...,1.72128,NaN,0.92685,0.00000,35.58424,12.04899,NaN,7.592766,782.797093,11.905930
1566,10.364485,0.064013,0.011423,0.002684,0.007113,1.921419,0.725000,1.470872,5.829607,0.554997,...,1.89140,NaN,0.48810,0.51861,36.15009,11.83649,NaN,10.791450,888.284510,14.138946


In [7]:
# meta_data.query('name in {}'.format(data.columns[numeric_vars]))
meta_data.query('name in {}'.format(list(data.columns[numeric_vars].values)))

,index,name,description,variable type,target
13,13,Viable_Cell_Density__g_over_L,Density of viable cells,independent,0
14,14,Product_Flux_Viable_Cells__mmol_over_gDCW_over_h,Cell-specific product flux,independent,0
15,15,Growth_Rate_Viable_Cells__1_over_h,Viable cell-specific growth rate,independent,0
16,16,Death_Rate_Avg__1_over_h,Cell-specific death rate,independent,0
17,17,Death_Rate_Avg_Viable_Cells__1_over_h,Viable cell-specific death rate,independent,0
...,...,...,...,...,...
115,115,Cap_PCV_End__percent,Packed cell volume capillary cell layer,independent,0
116,116,Cap_Dead_Cell_Layer_End__percent,Packed cell volume capillary dead cell layer,independent,0
117,117,Zeex9ieJAlt_end__g_L,Alternative assay product titer,independent,0
118,118,Zeex9ieJ_mAU_sec_end__area,Product titer peak area,independent,0


In [8]:
#prepare data for analysis
var_type_filter = [x in ['independent'] for x in meta_data['variable type']]
var_dtype_filter = (data.dtypes == 'float64') | (data.dtypes == 'int64')

numeric_vars = (var_type_filter & var_dtype_filter).values
numeric_x_data = data[data.columns[numeric_vars]]

#things to try to predict
y_data = data[data.columns[(meta_data['target'] == 1).values]]

#meta data about variables
meta_data = meta_data.query('name in {}'.format(list(data.columns[numeric_vars].values))).set_index('name')

In [9]:
#Variables which will be used to build the model
data.columns[numeric_vars].values

array(['Viable_Cell_Density__g_over_L',
       'Product_Flux_Viable_Cells__mmol_over_gDCW_over_h',
       'Growth_Rate_Viable_Cells__1_over_h', 'Death_Rate_Avg__1_over_h',
       'Death_Rate_Avg_Viable_Cells__1_over_h',
       'TRS_Flux_Viable_Cells__mmol_over_gDCW_over_h',
       'TRS_Flux__mmol_over_gDCW_over_h',
       'TRS_for_Other_Viable_Cells__mmol_over_gDCW_over_h',
       'sOUR_Viable_Cells__mmol_over_gDCW_over_h',
       'TRS_to_Maintenance_Flux__mmol_over_gDCW_over_h',
       'Est_Liquid_Solid_Yield__percent',
       'Est_Liquid_Liquid_Yield__percent', 'Est_Recovery__percent',
       'interval_day', 'interval_end_day', 'is_curated_interval',
       'Unadjusted_TRS_consumption_rate__g_over_gDCW_over_h',
       'TRS_for_Product__gTRS', 'TRS_for_Biomass__gTRS',
       'TRS_for_Glycerol__gTRS', 'TRS_for_Other__gTRS', 'TRS__g',
       'Interval_biomass__gDCW_over_L', 'TRS_consumed__g',
       'TRS_to_maint__g', 'Growth_Rate__hr_1',
       'Flux__g_product_over_gDCW_over_hr',
    

In [22]:
import functools

import numpy as np
from sklearn import compose
from sklearn import ensemble
from sklearn import impute
from sklearn import pipeline
from sklearn import preprocessing

classifier = ensemble.RandomForestClassifier(
  n_estimators=10,
  max_depth=10,
  min_samples_leaf=3,
)

numeric_transformer = pipeline.Pipeline([
  ('imputer', impute.SimpleImputer(strategy='median')),
  ('scaler', preprocessing.StandardScaler()),
])

transform_list = []
  # If there exist numerical columns
#if any(numerical_boolean):
transform_list.extend([
    ('numeric', numeric_transformer)
])

preprocessor = compose.ColumnTransformer(transform_list)

In [23]:
print(transform_list)

[('numeric', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())]))]


In [24]:
print(preprocessor)

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler',
                                                  StandardScaler())]))])


In [25]:
estimator = pipeline.Pipeline([
  ('preprocessor', preprocessor),
  ('classifier', classifier),
])

In [26]:
print(estimator)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]))])),
                ('classifier',
                 RandomForestClassifier(max_depth=10, min_samples_leaf=3,
                                        n_estimators=10))])
